In [1]:
import feature_extractor as fe 
from os import listdir
import scipy.io as spio
import numpy as np
import dataset

In [30]:
patient_number = 7
data, labels = dataset.get_data(patient_number)

307 number of points
1/307: extracting patient_7_197.mat
2/307: extracting patient_7_183.mat
3/307: extracting patient_7_154.mat
4/307: extracting patient_7_140.mat
5/307: extracting patient_7_168.mat
6/307: extracting patient_7_26.mat
7/307: extracting patient_7_32.mat
8/307: extracting patient_7_6.mat
9/307: extracting patient_7_222.mat
10/307: extracting patient_7_236.mat
11/307: extracting patient_7_237.mat
12/307: extracting patient_7_223.mat
13/307: extracting patient_7_7.mat
14/307: extracting patient_7_33.mat
15/307: extracting patient_7_27.mat
16/307: extracting patient_7_169.mat
17/307: extracting patient_7_141.mat
18/307: extracting patient_7_155.mat
19/307: extracting patient_7_182.mat
20/307: extracting patient_7_196.mat
21/307: extracting patient_7_180.mat


/usr/local/lib/python3.6/site-packages/scipy/signal/spectral.py:1785: UserWarning: nperseg = 256 is greater than input length  = 3, using nperseg = 3
  .format(nperseg, input_length))


22/307: extracting patient_7_194.mat
23/307: extracting patient_7_143.mat
24/307: extracting patient_7_157.mat
25/307: extracting patient_7_19.mat
26/307: extracting patient_7_31.mat
27/307: extracting patient_7_25.mat
28/307: extracting patient_7_5.mat
29/307: extracting patient_7_235.mat
30/307: extracting patient_7_221.mat
31/307: extracting patient_7_209.mat
32/307: extracting patient_7_208.mat
33/307: extracting patient_7_220.mat
34/307: extracting patient_7_234.mat
35/307: extracting patient_7_4.mat
36/307: extracting patient_7_24.mat
37/307: extracting patient_7_30.mat
38/307: extracting patient_7_18.mat
39/307: extracting patient_7_156.mat
40/307: extracting patient_7_142.mat
41/307: extracting patient_7_195.mat
42/307: extracting patient_7_181.mat
43/307: extracting patient_7_185.mat
44/307: extracting patient_7_191.mat
45/307: extracting patient_7_146.mat
46/307: extracting patient_7_152.mat
47/307: extracting patient_7_34.mat
48/307: extracting patient_7_20.mat
49/307: extra

260/307: extracting patient_7_160.mat
261/307: extracting patient_7_174.mat
262/307: extracting patient_7_189.mat
263/307: extracting patient_7_162.mat
264/307: extracting patient_7_176.mat
265/307: extracting patient_7_38.mat
266/307: extracting patient_7_10.mat
267/307: extracting patient_7_214.mat
268/307: extracting patient_7_200.mat
269/307: extracting patient_7_228.mat
270/307: extracting patient_7_229.mat
271/307: extracting patient_7_201.mat
272/307: extracting patient_7_215.mat
273/307: extracting patient_7_11.mat
274/307: extracting patient_7_39.mat
275/307: extracting patient_7_177.mat
276/307: extracting patient_7_163.mat
277/307: extracting patient_7_188.mat
278/307: extracting patient_7_198.mat
279/307: extracting patient_7_167.mat
280/307: extracting patient_7_173.mat
281/307: extracting patient_7_15.mat
282/307: extracting patient_7_29.mat
283/307: extracting patient_7_9.mat
284/307: extracting patient_7_239.mat
285/307: extracting patient_7_211.mat
286/307: extracting 

168/2400: extracting patient_7_1964.mat
169/2400: extracting patient_7_1970.mat
170/2400: extracting patient_7_433.mat
171/2400: extracting patient_7_355.mat
172/2400: extracting patient_7_1780.mat
173/2400: extracting patient_7_1958.mat
174/2400: extracting patient_7_2289.mat
175/2400: extracting patient_7_1794.mat
176/2400: extracting patient_7_369.mat
177/2400: extracting patient_7_2074.mat
178/2400: extracting patient_7_180.mat
179/2400: extracting patient_7_194.mat
180/2400: extracting patient_7_1569.mat
181/2400: extracting patient_7_2060.mat
182/2400: extracting patient_7_1541.mat
183/2400: extracting patient_7_802.mat
184/2400: extracting patient_7_2048.mat
185/2400: extracting patient_7_1227.mat
186/2400: extracting patient_7_1233.mat
187/2400: extracting patient_7_816.mat
188/2400: extracting patient_7_1555.mat
189/2400: extracting patient_7_625.mat
190/2400: extracting patient_7_143.mat
191/2400: extracting patient_7_157.mat
192/2400: extracting patient_7_631.mat
193/2400: e

...

1442/2400: extracting patient_7_67.mat
1443/2400: extracting patient_7_1463.mat
1444/2400: extracting patient_7_1305.mat
1445/2400: extracting patient_7_1488.mat
1446/2400: extracting patient_7_713.mat
1447/2400: extracting patient_7_2181.mat
1448/2400: extracting patient_7_707.mat
1449/2400: extracting patient_7_98.mat
1450/2400: extracting patient_7_2195.mat
1451/2400: extracting patient_7_1649.mat
1452/2400: extracting patient_7_1891.mat
1453/2400: extracting patient_7_2340.mat
1454/2400: extracting patient_7_2354.mat
1455/2400: extracting patient_7_1885.mat
1456/2400: extracting patient_7_1675.mat
1457/2400: extracting patient_7_1113.mat
1458/2400: extracting patient_7_288.mat
1459/2400: extracting patient_7_1107.mat
1460/2400: extracting patient_7_1661.mat
1461/2400: extracting patient_7_2368.mat
1462/2400: extracting patient_7_2383.mat
1463/2400: extracting patient_7_511.mat
1464/2400: extracting patient_7_1852.mat
1465/2400: extracting patient_7_277.mat
1466/2400: extracting pat

1656/2400: extracting patient_7_105.mat
1657/2400: extracting patient_7_893.mat
1658/2400: extracting patient_7_887.mat
1659/2400: extracting patient_7_139.mat
1660/2400: extracting patient_7_1249.mat
1661/2400: extracting patient_7_2026.mat
1662/2400: extracting patient_7_878.mat
1663/2400: extracting patient_7_2032.mat
1664/2400: extracting patient_7_1275.mat
1665/2400: extracting patient_7_1513.mat
1666/2400: extracting patient_7_850.mat
1667/2400: extracting patient_7_688.mat
1668/2400: extracting patient_7_844.mat
1669/2400: extracting patient_7_1507.mat
1670/2400: extracting patient_7_1261.mat
1671/2400: extracting patient_7_475.mat
1672/2400: extracting patient_7_1936.mat
1673/2400: extracting patient_7_1088.mat
1674/2400: extracting patient_7_313.mat
1675/2400: extracting patient_7_307.mat
1676/2400: extracting patient_7_1922.mat
1677/2400: extracting patient_7_461.mat
1678/2400: extracting patient_7_449.mat
1679/2400: extracting patient_7_2224.mat
1680/2400: extracting patient

1867/2400: extracting patient_7_607.mat
1868/2400: extracting patient_7_149.mat
1869/2400: extracting patient_7_2136.mat
1870/2400: extracting patient_7_1359.mat
1871/2400: extracting patient_7_2122.mat
1872/2400: extracting patient_7_968.mat
1873/2400: extracting patient_7_1403.mat
1874/2400: extracting patient_7_940.mat
1875/2400: extracting patient_7_798.mat
1876/2400: extracting patient_7_1365.mat
1877/2400: extracting patient_7_1371.mat
1878/2400: extracting patient_7_954.mat
1879/2400: extracting patient_7_13.mat
1880/2400: extracting patient_7_1417.mat
1881/2400: extracting patient_7_767.mat
1882/2400: extracting patient_7_773.mat
1883/2400: extracting patient_7_983.mat
1884/2400: extracting patient_7_997.mat
1885/2400: extracting patient_7_2334.mat
1886/2400: extracting patient_7_2320.mat
1887/2400: extracting patient_7_1629.mat
1888/2400: extracting patient_7_1167.mat
1889/2400: extracting patient_7_2308.mat
1890/2400: extracting patient_7_1601.mat
1891/2400: extracting patien

2077/2400: extracting patient_7_639.mat
2078/2400: extracting patient_7_177.mat
2079/2400: extracting patient_7_611.mat
2080/2400: extracting patient_7_2083.mat
2081/2400: extracting patient_7_605.mat
2082/2400: extracting patient_7_2097.mat
2083/2400: extracting patient_7_163.mat
2084/2400: extracting patient_7_1213.mat
2085/2400: extracting patient_7_188.mat
2086/2400: extracting patient_7_836.mat
2087/2400: extracting patient_7_1575.mat
2088/2400: extracting patient_7_1561.mat
2089/2400: extracting patient_7_2068.mat
2090/2400: extracting patient_7_822.mat
2091/2400: extracting patient_7_1207.mat
2092/2400: extracting patient_7_1549.mat
2093/2400: extracting patient_7_2040.mat
2094/2400: extracting patient_7_2054.mat
2095/2400: extracting patient_7_349.mat
2096/2400: extracting patient_7_1978.mat
2097/2400: extracting patient_7_1950.mat
2098/2400: extracting patient_7_1788.mat
2099/2400: extracting patient_7_2281.mat
2100/2400: extracting patient_7_413.mat
2101/2400: extracting pati

2287/2400: extracting patient_7_1228.mat
2288/2400: extracting patient_7_2047.mat
2289/2400: extracting patient_7_2053.mat
2290/2400: extracting patient_7_819.mat
2291/2400: extracting patient_7_158.mat
2292/2400: extracting patient_7_170.mat
2293/2400: extracting patient_7_2084.mat
2294/2400: extracting patient_7_616.mat
2295/2400: extracting patient_7_2090.mat
2296/2400: extracting patient_7_602.mat
2297/2400: extracting patient_7_1599.mat
2298/2400: extracting patient_7_164.mat
2299/2400: extracting patient_7_1360.mat
2300/2400: extracting patient_7_1406.mat
2301/2400: extracting patient_7_945.mat
2302/2400: extracting patient_7_951.mat
2303/2400: extracting patient_7_16.mat
2304/2400: extracting patient_7_789.mat
2305/2400: extracting patient_7_1412.mat
2306/2400: extracting patient_7_1374.mat
2307/2400: extracting patient_7_979.mat
2308/2400: extracting patient_7_2133.mat
2309/2400: extracting patient_7_2127.mat
2310/2400: extracting patient_7_1348.mat
2311/2400: extracting patien

In [31]:
print(data.shape)
print(labels.shape)

train_data, train_labels, val_data, val_labels = dataset.train_val_split(data, labels, .7)

print(train_data.shape)
print(train_labels.shape)
print(val_data.shape)
print(val_labels.shape)


(614, 24)
(614,)
(429, 24)
(429,)
(185, 24)
(185,)


In [40]:
from sklearn.svm import SVC
clf = SVC(C=100000, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=.00000000000001, kernel='rbf',
    max_iter=1000000, probability=True, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
clf.fit(train_data, train_labels)
y_pred = clf.predict(val_data)

from sklearn import metrics
print("training")
print(metrics.classification_report(clf.predict(train_data), train_labels))
print(metrics.roc_auc_score(clf.predict(train_data), train_labels))
print("\nvalidation")
print(metrics.classification_report(y_pred, val_labels))
print(metrics.roc_auc_score(y_pred, val_labels))
print("\n")

training
             precision    recall  f1-score   support

        0.0       0.95      0.92      0.94       223
        1.0       0.92      0.95      0.93       206

avg / total       0.94      0.93      0.93       429

0.935369410945187

validation
             precision    recall  f1-score   support

        0.0       0.88      0.81      0.84       100
        1.0       0.80      0.87      0.83        85

avg / total       0.84      0.84      0.84       185

0.8402941176470589




In [41]:
path = "data/patient_"+str(patient_number)+"/test/"
files = listdir(path)
print(str(len(files))+" number of points")
for i, file in enumerate(files):
    if file == ".DS_Store":
        continue
    print(str(i+1)+"/"+str(len(files))+": extracting "+file)
    file_data = spio.loadmat(path+file)["data"]
    file_data[np.isnan(file_data)] = 0
    vec = fe.extract_feature(file_data)
    
    filename = file[:-4].replace("test_", "")
    score = clf.predict_proba(vec.reshape(1,vec.shape[0]))[0,1]
    pred = [filename, score]
    import csv
    with open('patient.csv', 'a') as csvfile:
        spamwriter = csv.writer(csvfile, delimiter=',',
                                quotechar='|', quoting=csv.QUOTE_MINIMAL)
        spamwriter.writerow(pred)


4524 number of points
1/4524: extracting patient_7_test_3865.mat
2/4524: extracting patient_7_test_783.mat
3/4524: extracting patient_7_test_1914.mat
4/4524: extracting patient_7_test_1900.mat
5/4524: extracting patient_7_test_797.mat
6/4524: extracting patient_7_test_3871.mat
7/4524: extracting patient_7_test_967.mat
8/4524: extracting patient_7_test_3681.mat
9/4524: extracting patient_7_test_3859.mat
10/4524: extracting patient_7_test_1096.mat
11/4524: extracting patient_7_test_1928.mat
12/4524: extracting patient_7_test_1082.mat
13/4524: extracting patient_7_test_3695.mat
14/4524: extracting patient_7_test_973.mat
15/4524: extracting patient_7_test_740.mat
16/4524: extracting patient_7_test_2560.mat
17/4524: extracting patient_7_test_998.mat
18/4524: extracting patient_7_test_4111.mat
19/4524: extracting patient_7_test_1069.mat
20/4524: extracting patient_7_test_3118.mat
21/4524: extracting patient_7_test_2206.mat
22/4524: extracting patient_7_test_2212.mat


/usr/local/lib/python3.6/site-packages/scipy/signal/spectral.py:1785: UserWarning: nperseg = 256 is greater than input length  = 3, using nperseg = 3
  .format(nperseg, input_length))


23/4524: extracting patient_7_test_754.mat
24/4524: extracting patient_7_test_2574.mat
25/4524: extracting patient_7_test_4105.mat
26/4524: extracting patient_7_test_1055.mat
27/4524: extracting patient_7_test_3642.mat
28/4524: extracting patient_7_test_3124.mat
29/4524: extracting patient_7_test_1733.mat
30/4524: extracting patient_7_test_1727.mat
31/4524: extracting patient_7_test_3130.mat
32/4524: extracting patient_7_test_768.mat
33/4524: extracting patient_7_test_2548.mat
34/4524: extracting patient_7_test_3656.mat
35/4524: extracting patient_7_test_4139.mat
36/4524: extracting patient_7_test_1041.mat
37/4524: extracting patient_7_test_581.mat
38/4524: extracting patient_7_test_2979.mat
39/4524: extracting patient_7_test_595.mat
40/4524: extracting patient_7_test_1294.mat
41/4524: extracting patient_7_test_2945.mat
42/4524: extracting patient_7_test_3483.mat
43/4524: extracting patient_7_test_2789.mat
44/4524: extracting patient_7_test_2951.mat
45/4524: extracting patient_7_test_3

209/4524: extracting patient_7_test_4514.mat
210/4524: extracting patient_7_test_1478.mat
211/4524: extracting patient_7_test_351.mat
212/4524: extracting patient_7_test_2171.mat
213/4524: extracting patient_7_test_4500.mat
214/4524: extracting patient_7_test_4266.mat
215/4524: extracting patient_7_test_3509.mat
216/4524: extracting patient_7_test_2617.mat
217/4524: extracting patient_7_test_437.mat
218/4524: extracting patient_7_test_2824.mat
219/4524: extracting patient_7_test_3284.mat
220/4524: extracting patient_7_test_1493.mat
221/4524: extracting patient_7_test_1487.mat
222/4524: extracting patient_7_test_3290.mat
223/4524: extracting patient_7_test_2830.mat
224/4524: extracting patient_7_test_4299.mat
225/4524: extracting patient_7_test_2818.mat
226/4524: extracting patient_7_test_386.mat
227/4524: extracting patient_7_test_392.mat
228/4524: extracting patient_7_test_3455.mat
229/4524: extracting patient_7_test_2993.mat
230/4524: extracting patient_7_test_1242.mat
231/4524: extr

403/4524: extracting patient_7_test_1446.mat
404/4524: extracting patient_7_test_9.mat
405/4524: extracting patient_7_test_3251.mat
406/4524: extracting patient_7_test_3906.mat
407/4524: extracting patient_7_test_838.mat
408/4524: extracting patient_7_test_1877.mat
409/4524: extracting patient_7_test_186.mat
410/4524: extracting patient_7_test_192.mat
411/4524: extracting patient_7_test_1863.mat
412/4524: extracting patient_7_test_3912.mat
413/4524: extracting patient_7_test_804.mat
414/4524: extracting patient_7_test_3084.mat
415/4524: extracting patient_7_test_1693.mat
416/4524: extracting patient_7_test_90.mat
417/4524: extracting patient_7_test_1687.mat
418/4524: extracting patient_7_test_84.mat
419/4524: extracting patient_7_test_3090.mat
420/4524: extracting patient_7_test_4099.mat
421/4524: extracting patient_7_test_810.mat
422/4524: extracting patient_7_test_4072.mat
423/4524: extracting patient_7_test_2403.mat
424/4524: extracting patient_7_test_623.mat
425/4524: extracting pa

...

3656/4524: extracting patient_7_test_1075.mat
3657/4524: extracting patient_7_test_3662.mat
3658/4524: extracting patient_7_test_984.mat
3659/4524: extracting patient_7_test_3104.mat
3660/4524: extracting patient_7_test_1713.mat
3661/4524: extracting patient_7_test_2232.mat
3662/4524: extracting patient_7_test_4125.mat
3663/4524: extracting patient_7_test_2554.mat
3664/4524: extracting patient_7_test_3892.mat
3665/4524: extracting patient_7_test_774.mat
3666/4524: extracting patient_7_test_4131.mat
3667/4524: extracting patient_7_test_2540.mat
3668/4524: extracting patient_7_test_760.mat
3669/4524: extracting patient_7_test_3886.mat
3670/4524: extracting patient_7_test_1049.mat
3671/4524: extracting patient_7_test_2226.mat
3672/4524: extracting patient_7_test_3138.mat
3673/4524: extracting patient_7_test_953.mat
3674/4524: extracting patient_7_test_3879.mat
3675/4524: extracting patient_7_test_947.mat
3676/4524: extracting patient_7_test_1908.mat
3677/4524: extracting patient_7_test_19

3842/4524: extracting patient_7_test_603.mat
3843/4524: extracting patient_7_test_2423.mat
3844/4524: extracting patient_7_test_4052.mat
3845/4524: extracting patient_7_test_2345.mat
3846/4524: extracting patient_7_test_165.mat
3847/4524: extracting patient_7_test_1894.mat
3848/4524: extracting patient_7_test_830.mat
3849/4524: extracting patient_7_test_824.mat
3850/4524: extracting patient_7_test_2392.mat
3851/4524: extracting patient_7_test_1843.mat
3852/4524: extracting patient_7_test_98.mat
3853/4524: extracting patient_7_test_4085.mat
3854/4524: extracting patient_7_test_3932.mat
3855/4524: extracting patient_7_test_818.mat
3856/4524: extracting patient_7_test_4091.mat
3857/4524: extracting patient_7_test_3926.mat
3858/4524: extracting patient_7_test_1857.mat
3859/4524: extracting patient_7_test_2386.mat
3860/4524: extracting patient_7_test_3098.mat
3861/4524: extracting patient_7_test_4278.mat
3862/4524: extracting patient_7_test_3517.mat
3863/4524: extracting patient_7_test_2609

4029/4524: extracting patient_7_test_388.mat
4030/4524: extracting patient_7_test_2816.mat
4031/4524: extracting patient_7_test_4297.mat
4032/4524: extracting patient_7_test_1489.mat
4033/4524: extracting patient_7_test_2180.mat
4034/4524: extracting patient_7_test_2194.mat
4035/4524: extracting patient_7_test_4283.mat
4036/4524: extracting patient_7_test_2619.mat
4037/4524: extracting patient_7_test_439.mat
4038/4524: extracting patient_7_test_4268.mat
4039/4524: extracting patient_7_test_3507.mat
4040/4524: extracting patient_7_test_1310.mat
4041/4524: extracting patient_7_test_1476.mat
4042/4524: extracting patient_7_test_3261.mat
4043/4524: extracting patient_7_test_3275.mat
4044/4524: extracting patient_7_test_1462.mat
4045/4524: extracting patient_7_test_1304.mat
4046/4524: extracting patient_7_test_3513.mat
4047/4524: extracting patient_7_test_2625.mat
4048/4524: extracting patient_7_test_405.mat
4049/4524: extracting patient_7_test_4254.mat
4050/4524: extracting patient_7_test_

4220/4524: extracting patient_7_test_1701.mat
4221/4524: extracting patient_7_test_2208.mat
4222/4524: extracting patient_7_test_3116.mat
4223/4524: extracting patient_7_test_996.mat
4224/4524: extracting patient_7_test_3670.mat
4225/4524: extracting patient_7_test_1067.mat
4226/4524: extracting patient_7_test_3843.mat
4227/4524: extracting patient_7_test_2585.mat
4228/4524: extracting patient_7_test_1932.mat
4229/4524: extracting patient_7_test_1926.mat
4230/4524: extracting patient_7_test_3857.mat
4231/4524: extracting patient_7_test_2591.mat
4232/4524: extracting patient_7_test_969.mat
4233/4524: extracting patient_7_test_1098.mat
4234/4524: extracting patient_7_test_799.mat
4235/4524: extracting patient_7_test_941.mat
4236/4524: extracting patient_7_test_955.mat
4237/4524: extracting patient_7_test_943.mat
4238/4524: extracting patient_7_test_957.mat
4239/4524: extracting patient_7_test_3869.mat
4240/4524: extracting patient_7_test_1918.mat
4241/4524: extracting patient_7_test_1930

4412/4524: extracting patient_7_test_1689.mat
4413/4524: extracting patient_7_test_2380.mat
4414/4524: extracting patient_7_test_2394.mat
4415/4524: extracting patient_7_test_1845.mat
4416/4524: extracting patient_7_test_4083.mat
4417/4524: extracting patient_7_test_3934.mat
4418/4524: extracting patient_7_test_822.mat
4419/4524: extracting patient_7_test_1879.mat
4420/4524: extracting patient_7_test_188.mat
4421/4524: extracting patient_7_test_836.mat
4422/4524: extracting patient_7_test_3908.mat
4423/4524: extracting patient_7_test_7.mat
4424/4524: extracting patient_7_test_1448.mat
4425/4524: extracting patient_7_test_2141.mat
4426/4524: extracting patient_7_test_361.mat
4427/4524: extracting patient_7_test_3539.mat
4428/4524: extracting patient_7_test_4256.mat
4429/4524: extracting patient_7_test_407.mat
4430/4524: extracting patient_7_test_2627.mat
4431/4524: extracting patient_7_test_4242.mat
4432/4524: extracting patient_7_test_413.mat
4433/4524: extracting patient_7_test_2633.m